In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
FOLDERNAME = "Colab\ Notebooks/WeHelp/"

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/$FOLDERNAME

/content/drive/MyDrive/Colab Notebooks/WeHelp


In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from PIL import Image
import pandas as pd
import numpy as np

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print(device)

cuda


In [ ]:
# Paths to the data directories
data_dir = 'data/handwriting/'
augmented_images_dir = os.path.join(data_dir, 'augmented_images/augmented_images1')
combined_test_dir = os.path.join(data_dir, 'handwritten-english-characters-and-digits/combined_folder/test')

In [ ]:
SIZE = 28
transform = T.Compose([
    T.Resize((SIZE, SIZE)),
    T.ToTensor(),
    T.Grayscale(num_output_channels=1),
    T.Normalize((0.5,), (0.5,))
])

In [ ]:
BATCH = 32
train_data = ImageFolder(augmented_images_dir, transform=transform)
val_data = ImageFolder(combined_test_dir, transform=transform)

mini_trains = DataLoader(train_data, batch_size=BATCH, shuffle=True, num_workers=4)
mini_vals = DataLoader(val_data, batch_size=BATCH, num_workers=4)

NUM_TRAIN = len(train_data)
NUM_VAL = len(val_data)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
model = nn.Sequential(
    # N x 1 x 28 x 28
    nn.Conv2d(1, 32, 3, 1, 1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),  # N x 32 x 14 x 14

    nn.Conv2d(32, 64, 3, 1, 1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),  # N x 64 x 7 x 7

    nn.Conv2d(64, 128, 3, 1, 1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),  # N x 128 x 3 x 3

    nn.Flatten(),
    nn.Linear(128 * 3 * 3, 512),
    nn.ReLU(),
    nn.Linear(512, 62)
)
model = model.to(device)

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
NUM_EPOCH = 3
PRINT_EVERY = 20

In [ ]:
def val(mini_vals, model, device):
  model.eval()
  with torch.no_grad():
    total = 0
    for x, y in mini_vals:
      x = x.to(device)
      y = y.to(device)
      scores = model(x)
      predictions = scores.argmax(axis=1)
      acc = predictions.eq(y).sum().item()
      total += acc
    print("Total Acc:", total / NUM_VAL)

In [ ]:
def train(mini_trains, model, loss_function, optimizer, device, mini_vals):
  for epoch in range(NUM_EPOCH):
    count = 0
    for count, (x, y) in enumerate(mini_trains):
      model.train()
      x = x.to(device)
      y = y.to(device)
      scores = model(x)
      loss = loss_function(scores, y)
      if count % PRINT_EVERY == 0:
        print('Training Loss:', loss.item(), end=' / ')
        val(mini_vals, model, device)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

In [ ]:
train(mini_trains, model, loss_function, optimizer, device, mini_vals)

Training Loss: 4.229940414428711 / Total Acc: 0.016129032258064516
Training Loss: 4.1727776527404785 / Total Acc: 0.021994134897360705
Training Loss: 4.047776222229004 / Total Acc: 0.05571847507331378
Training Loss: 3.894242525100708 / Total Acc: 0.04838709677419355
Training Loss: 4.036081790924072 / Total Acc: 0.06744868035190615
Training Loss: 3.8439884185791016 / Total Acc: 0.0689149560117302
Training Loss: 3.328083038330078 / Total Acc: 0.17008797653958943
Training Loss: 3.317216634750366 / Total Acc: 0.22287390029325513
Training Loss: 2.92086124420166 / Total Acc: 0.14809384164222875
Training Loss: 2.6082441806793213 / Total Acc: 0.2727272727272727
Training Loss: 2.7096822261810303 / Total Acc: 0.35043988269794724
Training Loss: 2.108665943145752 / Total Acc: 0.43255131964809385
Training Loss: 1.822569727897644 / Total Acc: 0.46187683284457476
Training Loss: 1.9072309732437134 / Total Acc: 0.533724340175953
Training Loss: 1.2989475727081299 / Total Acc: 0.5894428152492669
Training